# はじめに
業務でword2vecを用いる必要性が出てきたため,まずword2vecで実際に遊んでみることにします.  
遊んでみたあとには理論を勉強して派生であるdoc2vec(paragraph2vec)も触れたいと思います.

word2vecはpythonのgensim: https://radimrehurek.com/gensim/models/word2vec.html ライブラリを用いる. また本実験では基本的にpython3系を利用する.  

すでに学習済みモデルを用いている場合は実験へ  
# 前処理
wikipediaのデータを手に入れる
```zsh
% curl https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2 -o jawiki-latest-pages-articles.xml.bz2
```


手に入れたデータがテキストでないのでテキストに変換する.  
変換ソフトとしてWikiExtractor.pyをgithubから落としてきて利用する.  
```zsh
% git clone https://github.com/attardi/wikiextractor
% python3 wikiextractor/WikiExtractor.py jawiki-latest-pages-articles.xml.bz2
```


上記プログラムを動かすとtext/以下に複数ファイルが生成されているので一つのファイルにまとめる
```zsh
% cat text/*/* > jawiki.txt
```

word2vecに読み込ませるために単語単位にセグメンテーションをする必要がある.  
今回はMeCab及びmecab-ipadic-NEologdの辞書を用いて行った.  
```zsh 
% mecab -Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd jawiki.txt -p ddata.txt
```
これで学習データの準備は整った

In [9]:
!head ddata.txt

< doc id =" 5 " url =" https :// ja . wikipedia . org / wiki ? curid = 5 " title =" アンパサンド "> 
アンパサンド 

アンパサンド (, &) と は 「 … と … 」 を 意味 する 記号 で ある 。 ラテン語 の の 合字 で 、 Trebuchet MS フォント で は 、 と 表示 さ れ " et " の 合字 で ある こと が 容易 に わかる 。 ampersa 、 すなわち " and per se and "、 その 意味 は " and [ the symbol which ] by itself [ is ] and " で ある 。 

その 使用 は 1世紀 に 遡る こと が でき ( 1 )、 5世紀 中葉 ( 2 , 3 ) から 現代 ( 4 - 6 ) に 至る まで の 変遷 が わかる 。 
Z に 続く ラテン文字 アルファベット の 27 字 目 と さ れ た 時期 も ある 。 

アンパサンド と 同じ 役割 を 果たす 文字 に 「 の et 」 と 呼ば れる 、 数字 の 「 7 」 に 似 た 記号 が あっ た (, U + 204 A )。 この 記号 は 現在 も ゲール文字 で 使わ れ て いる 。 



# 実験

## word2vec で遊ぶ

In [5]:
from gensim.models import word2vec,Word2Vec

学習方法(今回は省略した)
```python
data = word2vec.Text8Corpus("ddata.txt")
model = word2vec.Word2Vec(data,size=200) #sizeは次元数
model.save("w2v_wikipedia")
```

In [47]:
model = Word2Vec.load("MODEL/w2v")

In [38]:
#自前の自然言語処理ライブラリ
from load_datasets import *

その入力単語(列)と類似した単語候補を出力

In [39]:
input_text = split_word(input("input text: "),True).strip().split(" ")
model.most_similar(input_text)

input text: 佐倉綾音


[('茅野愛衣', 0.8792809844017029),
 ('真田アサミ', 0.8551074266433716),
 ('日笠陽子', 0.8525534272193909),
 ('伊瀬茉莉也', 0.8524771928787231),
 ('東山奈央', 0.8511958122253418),
 ('名塚佳織', 0.84943687915802),
 ('内田真礼', 0.8488936424255371),
 ('下野紘', 0.8487714529037476),
 ('福山潤', 0.8474154472351074),
 ('三森すずこ', 0.8453586101531982)]

"イスラム教" + "旧約聖書" - "コーラン" == ??? 

In [48]:
model.most_similar(positive=["イスラム教", "旧約聖書"], negative=["コーラン"])

[('ヒンドゥー教', 0.6836835145950317),
 ('ユダヤ教', 0.6696462035179138),
 ('キリスト教', 0.6457203030586243),
 ('キリスト教徒', 0.6282202005386353),
 ('イスラーム', 0.6207655668258667),
 ('ムスリム', 0.6194979548454285),
 ('イスラム教徒', 0.6105567216873169),
 ('ゾロアスター教', 0.6098810434341431),
 ('シーア派', 0.6076986193656921),
 ('正教会', 0.594813346862793)]

## ついでにfasttextでも遊ぶ. 
fasttextの導入及び学習方法について
```zsh
% git clone https://github.com/facebookresearch/fastText.git
% cd fastText
% make
% ./fasttext skipgram -input ../ddata.txt -output fmodel -dim 200 
```

せっかくなので学習済みモデルを使用して遊ぶことにした.  
学習済みモデルは以下のサイトから拝借しました.
- [Qita- fastTextの学習済みモデルを公開しました](http://qiita.com/Hironsan/items/513b9f93752ecee9e670):wikipedia
- [pixivnovel2vec](https://github.com/pixiv/pixivnovel2vec/releases):pixiv小説

In [260]:
from gensim.models.wrappers.fasttext import FastText
import gensim
fmodel = gensim.models.KeyedVectors.load_word2vec_format('MODEL/model.vec', binary=False)
#http://marmarossa.hatenablog.com/entry/2017/02/24/021331
#fmodel = FastText.load_fasttext_format('MODEL/model')

In [61]:
input_text = split_word(input("input text: "),True).strip().split(" ")
fmodel.most_similar(input_text)

input text: 佐倉綾音


[('種田梨沙', 0.6734929084777832),
 ('花江夏樹', 0.6484917402267456),
 ('大亀あすか', 0.6433695554733276),
 ('内田真礼', 0.6406412124633789),
 ('矢作紗友里', 0.6399767994880676),
 ('大原さやか', 0.6346932053565979),
 ('金元寿子', 0.6332128047943115),
 ('喜多村英梨', 0.6319013237953186),
 ('佐藤利奈', 0.6254490613937378),
 ('後藤邑子', 0.6248806118965149)]

コーラン: イスラム教の聖典  
イスラム教: 宗教の一つ  
旧約聖書: キリスト教の正典  
イスラム教 - コーラン + 旧約聖書 = ????  

In [62]:
fmodel.most_similar(positive=["イスラム教", "旧約聖書"], negative=["コーラン"])

[('キリスト教', 0.6492341756820679),
 ('ユダヤ教', 0.6367957592010498),
 ('キリスト教徒', 0.5953476428985596),
 ('新約聖書', 0.5876226425170898),
 ('マロン典礼カトリック教会', 0.5842365026473999),
 ('霊的キリスト教', 0.5791678428649902),
 ('聖書', 0.5702466368675232),
 ('キリスト', 0.5661367774009705),
 ('エチオピア正教会', 0.5649876594543457),
 ('ヘブライ語聖書', 0.5622918605804443)]

In [81]:
input_text = split_word(input("input text: "),True).strip().split(" ")
fmodel.most_similar(input_text)

input text: かわいい


[('可愛い', 0.7294535636901855),
 ('かわいい女', 0.670622706413269),
 ('女の子', 0.6256632208824158),
 ('可愛らしく', 0.6034404635429382),
 ('可愛らしい', 0.6003689765930176),
 ('かわいく', 0.5996170043945312),
 ('かわいらしく', 0.5813488960266113),
 ('おしゃれ', 0.5782523155212402),
 ('大人っぽい', 0.5750479698181152),
 ('かわいらしい', 0.5676910281181335)]

## fasttext -pixiv小説モデルで遊んでみる

In [116]:
from gensim.models.wrappers import FastText
ffmodel = FastText.load_fasttext_format('MODEL/pixiv_novel/fasttext-model')

In [248]:
input_text = split_word(input("input text: "),True).strip().split(" ")
ffmodel.most_similar(input_text)

input text: かわいい


[('可愛い', 0.8984634876251221),
 ('Cawaii!', 0.846759021282196),
 ('ざとかわいい', 0.8390275239944458),
 ('エロかわいい', 0.7630234956741333),
 ('ぇかわいい', 0.7591103315353394),
 ('ずるい', 0.7465609908103943),
 ('カワイイー', 0.7461790442466736),
 ('ぁわいい', 0.7455922961235046),
 ('ーカワイイ', 0.744534969329834),
 ('かっこいい', 0.7317109704017639)]

In [249]:
input_text = split_word(input("input text: "),True).strip().split(" ")
ffmodel.most_similar(input_text)

input text: 佐倉綾音


[('伊藤かな恵', 0.8857293725013733),
 ('佐藤利奈', 0.8840634822845459),
 ('花澤香菜', 0.8804652094841003),
 ('水瀬いのり', 0.8723459839820862),
 ('今井麻美', 0.8675230741500854),
 ('早見沙織', 0.8672717213630676),
 ('岡本信彦', 0.866820216178894),
 ('日笠陽子', 0.8660044074058533),
 ('茅野愛衣', 0.8659030795097351),
 ('金元寿子', 0.8609911799430847)]

In [82]:
#pmodel = gensim.models.KeyedVectors.load_word2vec_format('MODEL/pixiv_novel/fasttext-model.vec', binary=False)

分かち書きをしなくて文字列を入力として候補を出力できる. 実験してみて似た雰囲気の言葉を返しているわけでもなさそう.. 
```python
input_text = input('input text: ')
print("-------------Not 分かち書き-------------")
for x in pmodel.most_similar(input_text):
    print(x[0],x[1])
print("--------------分かち書き(NE)----------------")
input_text0 = split_word(input_text,True).strip().split(" ")
print(input_text0)
for x in pmodel.most_similar(positive=input_text0):
    print(x[0],x[1])
print("--------------分かち書き(default)----------------")
input_text1 = split_word(input_text,False).strip().split(" ")
print(input_text1)
for x in pmodel.most_similar(input_text1):
    print(x[0],x[1])
```

## doc2vec-pixiv小説モデルで遊んでみる  
doc2vecは辞書に含まれていない単語には対応できない(要出典)

In [101]:
## doc2vec でも遊ぶ
from gensim.models import doc2vec
pdmodel = doc2vec.Doc2Vec.load("MODEL/pixiv_novel/doc2vec.model")

In [241]:
input_text = input("input text: ")
pdmodel.most_similar(input_text)

input text: 図書館戦争


[('クロスオーバー', 0.5370278358459473),
 ('二次創作', 0.5299734473228455),
 ('R指定', 0.5223528146743774),
 ('ネタバレ', 0.5196671485900879),
 ('設定', 0.5082077980041504),
 ('ノベライズ', 0.4985283613204956),
 ('スノーボード', 0.49830979108810425),
 ('ゲーム版', 0.49589353799819946),
 ('再掲', 0.4945579469203949),
 ('原作', 0.4923849403858185)]

In [244]:
input_text = split_word(input("input text: "),False).strip().split(" ")
print("split>>",input_text)
pdmodel.most_similar(input_text)

input text: 図書館戦争
split>> ['図書館', '戦争']


[('幻想郷', 0.9157354235649109),
 ('学校', 0.9109996557235718),
 ('教会', 0.9004945158958435),
 ('聖域', 0.9004569053649902),
 ('江戸', 0.8955792188644409),
 ('地球', 0.8928031921386719),
 ('魔界', 0.8840538859367371),
 ('孤児院', 0.8830622434616089),
 ('天界', 0.8813377618789673),
 ('王都', 0.8804082870483398)]

In [246]:
print("input >>",input_text[0])
pdmodel.most_similar(input_text[0])

input >> 図書館


[('保健室', 0.9106063842773438),
 ('学校', 0.9057930707931519),
 ('王都', 0.905574381351471),
 ('道場', 0.9044901132583618),
 ('書庫', 0.9012769460678101),
 ('東京', 0.8968735337257385),
 ('事務所', 0.8929109573364258),
 ('城下', 0.8927860260009766),
 ('教会', 0.8926318287849426),
 ('食堂', 0.8922826647758484)]

# 所感
word2vecよりfasttextの方がほしい結果が出た気がする. ただ評価が難しい....  
fasttextはword2vecやdoc2vecと異なり辞書に含まれていない単語に対しても何かしらの候補単語を示すことはできる.  ただし,それがどういう感なのかはいまいちわからない..

word2vecとfasttextの大きな違いはfasttextに組み込まれているsubword(部分語)情報を用いてる点である.  

In [317]:
#fmodel = gensim.models.KeyedVectors.load_word2vec_format('MODEL/model.vec', binary=False)
def input_output(i,modelName):
    '''
    ffmodel: pixiv
    fmodel :wikipeda
    '''
    #i = input("input text: ")
    input_text = split_word(i,True).strip().split(" ")
    #print(input_text)
    

    #mecab-ipadic-neologd で分かち書きしてうまくいかなかった場合デフォルトの辞書で分かち書きして入力
    try :
        print("try: ",input_text)
        for x in modelName.most_similar(input_text):
            print(x)
    except KeyError:
        try:
            input_text = split_word(i,False).strip().split(" ")
            print("except: ",input_text)
            for x in modelName.most_similar(input_text):
                print(x)
        except KeyError:
            try:
                print("全部無理ぽ: ",i)
                for x in modelName.most_similar([i]):
                    print(x)
            except:
                print("----文字自体が辞書に含まれていませんでした----")

In [318]:
input_output("(*ﾟーﾟ)",ffmodel)

try:  ['(*ﾟーﾟ)']
("(*'", 0.762630820274353)
('(*>', 0.739618718624115)
('＼*)', 0.738576352596283)
("'*)", 0.7343478202819824)
('＾*)', 0.7328968048095703)
('(*/∀＼*)', 0.729537844657898)
('(*>∀<*)', 0.7287780046463013)
('(*´∇`)', 0.7278110980987549)
('(^▽^)', 0.7234911918640137)
('(*‘ω‘', 0.7228749990463257)


In [319]:
input_output("(*ﾟーﾟ)",fmodel)

try:  ['(*ﾟーﾟ)']
except:  ['(*', 'ﾟーﾟ', ')']
全部無理ぽ:  (*ﾟーﾟ)
----文字自体が辞書に含まれていませんでした----


In [321]:
input_output("(´･ω･｀)",ffmodel)

try:  ['(´･ω･｀)']
('バラ科', 0.374181866645813)
('植え込む', 0.3652542233467102)
('人為的', 0.35676005482673645)
('摘み取る', 0.35282132029533386)
('生存戦略', 0.3414807915687561)
('生存権', 0.3391748368740082)
('寄生植物', 0.3357234299182892)
('宿り木', 0.3337258994579315)
('幼体', 0.33261629939079285)
('病原菌', 0.33160269260406494)


In [320]:
input_output("(´･ω･｀)",fmodel)

try:  ['(´･ω･｀)']
except:  ['(´･', 'ω', '･｀)']
全部無理ぽ:  (´･ω･｀)
----文字自体が辞書に含まれていませんでした----


In [323]:
input_output("＼(^o^)／",ffmodel)

try:  ['＼(^o^)／']
('＼(^', 0.8958107233047485)
('(^o^)/', 0.8782684803009033)
('＼(^^)／', 0.8727320432662964)
('o(^o^)o', 0.863396406173706)
('^)／', 0.8564893007278442)
('／(^o^)＼', 0.8384049534797668)
('(￣｡￣;)', 0.8368974924087524)
('＼(＾', 0.8368598222732544)
('(^O^)／', 0.819638729095459)
('(;O;)', 0.8176270127296448)
